In [16]:
#!pip install -U sentence-transformers
#from sentence_transformers import SentenceTransformers, util
from sentence_transformers import util
from transformers import AutoTokenizer, AutoModel
import torch
import json
import time
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sbert-nli')
model = AutoModel.from_pretrained('jhgan/ko-sbert-nli')
#model = SentenceTransformer("Huffon/sentence-klue-roberta-base")

infile = 'q1.json'
outfile = 'a1.json'

with open(infile, encoding='UTF-8') as f:
    #json_data = json.load(f)
    json_data =json.loads(f.read())
    #lines = f.readlines()

    
'''
cont=""
for line in lines:
    cont = cont+line;

json_data = json.loads(cont)
'''
json_out = []
cnt = 0
for d in json_data:
    cnt = cnt+1
    start = time.time()
    problem_no = d['problem_no']
    task_no = d['task_no']
    doc_file = d['doc_file']
    question = d['question']
    paras = d['paras']
    sent = d['sent']
    para = d['para']
    pat = d['pat']
    
    
    query = question
    sentences = paras
    ####sentences.append(question)
    
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    encoded_input2 = tokenizer(query, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
        model_output2 = model(**encoded_input2)

    # Perform pooling. In this case, mean pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    query_embedding = mean_pooling(model_output2, encoded_input2['attention_mask'])

    #sentence_embeddings = model_output[:-1]
    #query_embedding = model_output[-1]

    top_k = min(3, len(sentences))
    cos_scores = util.pytorch_cos_sim(query_embedding, sentence_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    
    a = {}
    a['team_id']=""
    a['hash']=""
    a['problem_no']=problem_no
    a['task_no']=task_no
    
    a['doc_file']=doc_file
    a['time']=delta
    a['question'] = question
    a['para'] = para
    answer = []
    for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
        rank = str(i+1);
        paragraph = sentences[idx]
        ans={}
        ans['rank']=rank
        ans['paragraph']=paragraph
        answer.append(ans)
    a['answer']=answer
    a['sent'] = sent
    delta = str(time.time()-start)
    
    
    json_out.append(a);
    #if(cnt%500==0):
    if(cnt%5==0):
        out = open(outfile,'w')
        out.write(json.dumps(json_out, ensure_ascii=False, indent=4))
        out.close();
        
        print("\n")
        print(str(cnt)+"/"+str(len(json_data))+" "+str(delta)[:4]+"초");
        print(f"Question: {query}")
        #print(f"<입력 문장과 유사한 {top_k} 개의 문장>")

        for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
            print(f"[{i+1}] {sentences[idx][0:min(200,len(sentences[idx]))]} {'(유사도: {:.4f})'.format(score)}")

        





5/27983 15.초
Question: 영덕군 영해면 성내
[1]   농촌현장포럼 결과보고서영덕 예주고을(박승배 사무장)1. 예주권역의 마을 종합개발사업의 시작 예주권역 거점면소재지 마을종합개발사업은 총사업비 70억원(국비 49억원, 지방비 21억원)으로 오는 12월까지 예주생활관, 한마음광장, 등산로정비, 중심상가주차장, 삼각주공원, 담장정비 및 컨설팅, 홍보 마케팅, 교육, 정보화 사업 등 핵심사업구역을 지정해 사업이 (유사도: 0.5859)
[2] 함께하는 우리 농촌 운동절의 고장이며, 고려조의 대 사상가(성리학)이자 훌륭한 재상으로써 불사이군의 충절과 고려 문학을 대표하는 대 문인인 목은 이색선생을 배출한 곳이기도 합니다. 예주문화예술회관과 생활체육공원, 3.1의거탑, 이색목은기념관 등이 있으며, 이를 중심으로 괴시리 전통마을 등 유교문화권사업이 완성되면 전통문화와 충절을 배우는 산 교육의 장으로  (유사도: 0.5858)
[3]   농촌현장포럼 결과보고서 현장견학지 소개영덕옥계권역(이민석 위원장)영덕 옥계권역 농촌마을종합개발사업 개요 1. 위      치 : 경상북도영덕군 달산면 옥산리, 주응리, 흥기리2. 규      모 : 3개 법정리, 6개 행정리 (옥산1,2,3리, 주응1,2리, 흥기3리) 3. 면      적 : 1,935.8ha (농경지: 112.8ha, 임야: 1,82 (유사도: 0.5583)


10/27983 13.초
Question: ‘Halla Gold’) 같은 골드키위들이 계속해서 육성 보급되는 계기가 되었다(Kim 
[1] 녹색인 그린키위 밖에 없었기 때문에 국내에서도 ‘헤이워드’ 품종을 대체할 수 있는 품종개발에대한 연구만이 진행되었다(김, 2014). 1990년대에 농촌진흥청 국립원예특작과학원을 중심으로 교잡육종을 통하여 ‘보옥’, ‘제시그린’, ‘제시스위트’ 및 ‘보화’ 품종이 육성되었으나 농가재배는 극히 일부에서 이루지고 있다(Kim et al., 2007b; 국립종자 (유사도: 0.6030)
[2] 성과 세부 성과

KeyboardInterrupt: 